In [8]:
import sqlite3
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

In [9]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('actors',),
 ('directors',),
 ('writers',),
 ('movies',),
 ('movie_actors',),
 ('movie_directors',),
 ('movie_writers',)]

In [10]:
cursor.execute('PRAGMA table_info(movies)')
cursor.fetchall()

[(0, 'id', 'INT', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'studio', 'TEXT', 0, None, 0),
 (3, 'runtime', 'REAL', 0, None, 0),
 (4, 'description', 'TEXT', 0, None, 0),
 (5, 'release_date', 'NUM', 0, None, 0),
 (6, 'year', 'INT', 0, None, 0)]

In [11]:
movies_df = pd.read_sql('SELECT * FROM movies', conn)

In [12]:
movies_df[:2]

,id,title,studio,runtime,description,release_date,year
0,11784,!Women Art Revolution,Zeitgeist Films,83.0,"Through intimate interviews, art, and rarely s...",2011-06-01 00:00:00,2011
1,21806,#Horror,Lowland Pictures,90.0,You've got followers... Cyberbullying goes off...,2015-11-20 00:00:00,2015


In [ ]:
import pandas as pd

In [5]:
df = pd.read_csv('./movies_metadata.csv')

/Users/jeff/opt/miniconda3/envs/data_engineering/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
merged_df = movies_df.merge(df, on = ['title'])

In [14]:
merged_df[:2]

,id_x,title,studio,runtime_x,description,release_date_x,year,adult,belongs_to_collection,budget,...,production_countries,release_date_y,revenue,runtime_y,spoken_languages,status,tagline,video,vote_average,vote_count
0,11784,!Women Art Revolution,Zeitgeist Films,83.0,"Through intimate interviews, art, and rarely s...",2011-06-01 00:00:00,2011,False,NaN,0,...,[],2010-01-01,0.0,83.0,[],Released,40 years in the making!,False,4.3,2.0
1,21806,#Horror,Lowland Pictures,90.0,You've got followers... Cyberbullying goes off...,2015-11-20 00:00:00,2015,False,NaN,1500000,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2015-11-20,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Death is trending.,False,3.4,53.0


In [20]:
selected_cols = ['id_x', 'title', 'studio', 'runtime_x', 'description', 'release_date_x',
       'year', 'adult', 'revenue', 'budget', 'imdb_id', 
                 'popularity', 'tagline', 'vote_average', 'vote_count']

selected_df = merged_df[selected_cols] 

In [102]:
df_revenue = selected_df[selected_df['revenue'] != 0.0]

In [103]:
df_revenue.columns = ['id', 'title', 'studio', 'runtime', 'description', 'release_date',
       'year', 'adult', 'revenue', 'budget', 'imdb_id', 'popularity',
       'tagline', 'vote_average', 'vote_count']

In [38]:
cols = ['id', 'title', 'runtime', 'revenue', 'budget', 'popularity',
       'vote_average', 'vote_count']

movies_updated = df_revenue[cols]

In [43]:
profit = pd.to_numeric(movies_updated['revenue']) - pd.to_numeric(movies_updated['budget'])

In [48]:
df_rev = movies_updated.assign(profit = profit)

In [49]:
df_rev[:2]

,id,title,runtime,revenue,budget,popularity,vote_average,vote_count,profit
3,201,'71,100.0,1625847.0,0,7.080550,6.7,414.0,1625847.0
5,19696,...And Justice for All,119.0,33300000.0,0,6.291512,7.1,118.0,33300000.0


### Joining Tables

In [ ]:
[('actors',),
 ('directors',),
 ('writers',),
 ('movies',),
 ('movie_actors',),
 ('movie_directors',),
 ('movie_writers',)]

In [53]:
df_movie_actors = pd.read_sql('''SELECT movie_actors.*, actors.* FROM movie_actors 
            JOIN actors ON actors.id = movie_actors.actor_id
            ''', conn)

In [65]:
selected_actors = df_movie_actors[['movie_id', 'name']]

In [ ]:
df_movie_actors = pd.read_sql('''SELECT movie_actors.*, actors.* FROM movie_actors 
            JOIN actors ON actors.id = movie_actors.actor_id
            ''', conn)

In [60]:
df_movie_directors = pd.read_sql('''SELECT movie_directors.*, directors.* FROM movie_directors 
            JOIN directors ON directors.id = movie_directors.director_id
            ''', conn)

In [64]:
selected_directors = df_movie_directors[['movie_id', 'name']]

In [68]:
df_movie_writers = pd.read_sql('''SELECT movie_writers.*, writers.* FROM movie_writers 
            JOIN writers ON writers.id = movie_writers.writer_id
            ''', conn)

In [69]:
selected_movie_writers = df_movie_writers[['movie_id', 'name']]

In [71]:
selected_movie_writers[:2]

,movie_id,name
0,1,Alfred Gough
1,1,Miles Millar


In [ ]:
selected_movie_writers, selected_directors, selected_actors, 

In [87]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [125]:
import numpy as np
import re
# add_datepart(df_revenue, 'release_date', drop = False)

In [112]:
date_cols = ['id', 'title', 'release_date', 'release_Year', 'release_Month', 'release_Week', 'release_Day', 'release_Dayofweek', 'release_Dayofyear']

date_df = df_revenue[date_cols]

In [115]:
date_df.columns = ['movie_id', 'title', 'release_date', 'year', 'month', 'week', 'day', 'wday', 'day_of_year']

In [119]:
selected_date_df = date_df.drop(columns = ['title'])

In [123]:
date_df_idx = selected_date_df.reset_index(drop = True)

In [136]:
date_df_idx.shape

(5553, 8)

In [141]:
date_df_idx.shape

(5553, 8)

In [143]:
merged_movies_df = date_df_idx.merge(df_rev, left_on = 'movie_id', right_on = 'id')

In [186]:
merged_movies_df[:2]

,movie_id,release_date,year,month,week,day,wday,day_of_year,id,title,runtime,revenue,budget,popularity,vote_average,vote_count,profit
0,201,2015-02-27,2015,2,9,27,4,58,201,'71,100.0,1625847.0,0,7.080550,6.7,414.0,1625847.0
1,19696,1979-06-29,1979,6,26,29,4,180,19696,...And Justice for All,119.0,33300000.0,0,6.291512,7.1,118.0,33300000.0


In [148]:
date_df = merged_movies_df[['release_date', 'year', 'month', 'week', 'day', 'wday',
       'day_of_year']]

In [150]:
date_df[:2]

,release_date,year,month,week,day,wday,day_of_year
0,2015-02-27,2015,2,9,27,4,58
1,1979-06-29,1979,6,26,29,4,180


In [152]:
df_revenue_updated = merged_movies_df.assign(date_id = date_df.index)

In [187]:
selected_rev_df = df_revenue_updated[['id', 'title', 'runtime', 'date_id',
       'popularity', 'vote_average', 'vote_count', 'revenue', 'budget','profit']]

In [190]:
selected_rev_df[:2]

,id,title,runtime,date_id,popularity,vote_average,vote_count,revenue,budget,profit
0,201,'71,100.0,0,7.080550,6.7,414.0,1625847.0,0,1625847.0
1,19696,...And Justice for All,119.0,1,6.291512,7.1,118.0,33300000.0,0,33300000.0


In [162]:
date_df.loc[4]

release_date    1999-03-31 00:00:00
year                           1999
month                             3
week                             13
day                              31
wday                              2
day_of_year                      90
Name: 4, dtype: object

In [ ]:
date_df, selected_rev_df, selected_movie_writers, selected_directors, selected_actors

In [191]:
selected_rev_df.columns = ['id', 'title', 'runtime', 'release_date_id', 'popularity', 'vote_average', 
'vote_count', 'revenue', 'budget', 'profit']

In [164]:
import sqlite3
conn = sqlite3.connect('star_films.db')
cursor = conn.cursor()

In [170]:
date_df.to_sql('release_dates', conn, index = True, if_exists = 'replace', index_label = 'id')

In [192]:
selected_rev_df.to_sql('movies', conn, index = False, if_exists = 'replace')

In [198]:
selected_rev_df[10:22]

,id,title,runtime,release_date_id,popularity,vote_average,vote_count,revenue,budget,profit
10,8722,102 Dalmatians,100.0,10,7.668752,5.1,327.0,183611771.0,85000000,98611771.0
11,5602,12,180.0,11,6.975759,6.6,54.0,7537453.0,4000000,3537453.0
12,352,12 Rounds,107.0,12,13.591485,5.7,223.0,17280326.0,20000000,-2719674.0
13,19106,12 Years a Slave,134.0,13,30.316249,7.9,3787.0,187000000.0,20000000,167000000.0
14,18635,127 Hours,94.0,14,11.176797,7.0,2668.0,35692920.0,18000000,17692920.0
15,16883,13,97.0,15,14.679860,5.9,247.0,3317662.0,0,3317662.0
16,1316,13 Assassins,126.0,16,9.534232,7.1,362.0,17555141.0,6000000,11555141.0
17,28197,13 Going on 30,98.0,17,12.632595,6.3,1260.0,96455697.0,37000000,59455697.0
18,22378,13 Sins,92.0,18,9.067788,6.2,355.0,13809.0,5000000,-4986191.0
19,4797,13 Tzameti,93.0,19,2.384760,6.9,66.0,767311.0,0,767311.0
